## 超定方程组 
对于方程组Ra=y，R为n×m矩阵，如果R列满秩，且n>m。则方程组没有精确解，此时称方程组为超定方程组。   
1. 超定方程一般是不存在解的矛盾方程，比较常用的方法是最小二乘法。  
2. 曲线拟合是最小二乘法要解决的问题，实际上就是求以上超定方程组的最小二乘解的问题。  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(suppress = True)

In [ ]:
n = 3
m = 100
np.random.seed(5)
x = np.random.normal(0, 1, m * n).reshape(m, n)
np.random.seed(10)
y = np.dot(x, np.array(range(1, n + 1))) + np.random.normal(0, 0.1, m)

print(x[0], y[0])

## 最小二乘法 
假设数据集有m行n列，且满足m > n，那么这个方程组有m个方程和n个未知数，无法求解   

由于：  
$e_{i} = Y_{i} - \hat{Y_{i}}$  

$e_{i} = Y_{i} - (\sum_{j=1}^{n}\beta_{j}X_{ij})$  

$MSE = \frac{1}{m}\sum_{i=1}^{m}e_{i}^2 = \frac{1}{m}\sum_{1}^{m}(Y_{i} - \sum_{j=1}^{n}\beta_{j}X_{ij})^2$  

显然我们不太可能找到一组解让MSE等于0，但是我们可以求出一组解让MSE最小。  上面关于MSE的等式称作损失函数，我们求解方程组的任务也转为最小化损失函数的值。对n个beta求偏导，令偏导数等于零，则得到n个方程组且有n个未知数，方程组有唯一解如下：

$\beta = (X^TX)^{-1}X^TY$

通过numpy的矩阵乘法、求逆和转置来实现least_squares_method，并与numpy自带的线性最小二乘求解函数np.linalg.lstsq进行结果对比

In [ ]:
def add_bias(x):
    return np.concatenate((np.ones(x.shape[0]).reshape(-1,1), x), axis=1)


t = np.array(range(10)).reshape(5, 2)
print("The origin x is:\n", t, '\n')

t = add_bias(t)
print("The result is:\n", t, '\n')

In [ ]:
def least_squares_method(x, y):
    x = np.mat(add_bias(x))
    y = np.mat(y).T
    return np.array((x.T * x).I * x.T * y)

betas1 = least_squares_method(x, y)
betas2 = np.linalg.lstsq(add_bias(x), y)[0].reshape(-1, 1)
print(np.concatenate((betas1, betas2), axis=1))

##  什么是牛顿法
牛顿法是一种在实数域和复数域上近似求解方程的方法。方法使用函数f(x)的泰勒级数的前面几项来寻找方程f(y)=0的根。  
  
泰勒公式，如果$f(x)$在点$x=x_0$具有任意阶导数，则幂级数  
$f(x) = \large\frac{f(x_0)}{0!}\normalsize+\large\frac{f'(x_0)}{1!} \normalsize(x-x_0)+\large\frac{f''(x_0)}{2!} \normalsize(x-x_0)^2
+...+\large\frac{f^{(n)}(x_0)}{n!} \normalsize(x-x_0)^n + R_n(x)$  
x_0是初始值，x_1是x_0更新后的值  
近似认为：$f(x_1) = f(x_0) + (x_1 - x_0)f'(x_0) +(x_1 - x_0)^2\large\frac{f''(x_0)}{2}$  
两边求导：$f'(x_1) = f'(x_0) + (x_1 - x_0) {f''(x_0)}$  
令$f'(x_1) = 0$  
解方程，得到：$x_1 = x_0 - \large\frac{f'(x_0)}{f''(x_0)}$    
不断迭代，得到：$x_{n+1} = x_n - \large\frac{f'(x_n)}{f''(x_n)}$    
上式就是牛顿法的更新公式

## 矩阵除法
$AA^{-1} = E$  
$A左除B = A^{-1}B$  
$B右除A = BA^{-1}$

In [ ]:
a = np.matrix([2, 1, 4, -3]).reshape(2,2)
print("Matrix A is:\n", a, "\n")
print("Matrix A's inverse matrix is:\n", a.I, "\n")
print("Matrix A product its inverse matrix is:\n", a * a.I, "\n")

In [ ]:
b = np.matrix([1, 2])

In [ ]:
b * a.I

In [ ]:
a.I * b

## 利用牛顿法求解最小二乘问题
X为$m \times n$的矩阵，Y为$m \times 1$的矩阵，W为$n \times 1$的矩阵  
  
1.优化函数  
线性函数： $Y = XW = [\sum_{j}^{n}w_{j}x_{1j},\sum_{j}^{n}w_{j}x_{2j},...,\sum_{j}^{n}w_{j}x_{mj}]^T$  
误差函数： $ERR = \hat{Y} - Y$  
损失函数： $L = \large\frac{1}{m}\normalsize\sum_{i}^{m}err_{i}^2$  
为了求导方便，等价于： $L = \large\frac{1}{2}\normalsize\sum_{i}^{m}err_{i}^2$
  
2.一阶导数  
对$w_1, w_2...w_n$分别求偏导得到$n \times 1$矩阵：  
$[\large\frac{\partial L}{\partial w_1}, \frac{\partial L}{\partial w_2},...,\frac{\partial L}{\partial w_n}]$  
  
向量中的第a个元素等于：$\large\frac{\partial L}{\partial w_a} \normalsize = \sum_{i}^{m}err_{i}x_{ia}
= \sum_{i}^{m}(\hat{y_i} - y_i)x_{ia}$  
  
所以向量等于：
$[\sum_{i}^{m}err_{i}x_{i1}, \sum_{i}^{m}err_{i}x_{i2},...,\sum_{i}^{m}err_{i}x_{in}]
= X^T \times ERR = X^T \times (\hat{Y} - Y) = X^T \times (XW - Y)$
    
3.二阶导数  
对上述向量的元素分别求偏导得到$n \times n$阶方阵：  
$
  \begin{matrix}
   \large\frac{\partial^2 L}{\partial^2 w_1} & \large\frac{\partial^2 L}{\partial w_1\partial w_2} & ... & \large\frac{\partial^2 L}{\partial w_1\partial w_n} \\
   \large\frac{\partial^2 L}{\partial w_2\partial w_1} & \large\frac{\partial^2 L}{\partial w_2\partial w_2} & ... & \large\frac{\partial^2 L}{\partial w_2\partial w_n} \\
   ... & ... & ... & ...\\
   \large\frac{\partial^2 L}{\partial w_n\partial w_1} & \large\frac{\partial^2 L}{\partial w_n\partial w_2} & ... & \large\frac{\partial^2 L}{\partial^2 w_n} \\
  \end{matrix} \
$

这个方阵也被称为黑塞矩阵（Hessian Matrix），又译作海森矩阵、海瑟矩阵、海塞矩阵等  
矩阵中的第(a, b)个元素等于： 
$\large\frac{\partial^2 L}{\partial w_a \partial w_b} \normalsize
= \large\frac{\partial{\sum_{i}^{m}(\sum_{j}^{n}w_{j}x_{ij} - y_{i})x_{ia}}}{\partial w_b} 
\normalsize = \sum_{i}^{m}x_{ia}x_{ib}$  
  
所以矩阵H等于：  
$
  \begin{matrix}
   \sum_{i}^{m}x_{i1}x_{i1} & \sum_{i}^{m}x_{i1}x_{i2} & ... & \sum_{i}^{m}x_{i1}x_{in} \\
   \sum_{i}^{m}x_{i2}x_{i1} & \sum_{i}^{m}x_{i2}x_{i2} & ... & \sum_{i}^{m}x_{i2}x_{in} \\
   ... & ... & ... & ... \\
   \sum_{i}^{m}x_{in}x_{i1} & \sum_{i}^{m}x_{in}x_{i2} & ... & \sum_{i}^{m}x_{in}x_{in} \\
  \end{matrix}
$  
  
$ = X^T X$  
  
4.迭代公式  
$W_{new} = W - \large\frac{f(W)}{f'(W)}\normalsize= W - (X^T X)^{-1} \times X^T(XW - Y)$  

In [ ]:
def newton_method(x, y):
    x = np.matrix(add_bias(x))
    m, n = x.shape
    y = np.matrix(y).reshape(m, 1)
    weights = np.matrix(np.random.normal(0, 0.01, n)).reshape(n, 1)
    return weights - (x.T * x).I * x.T * (x * weights - y)

In [ ]:
newton_method(x, y)

In [ ]:
[[ 0.00854145  0.00854145]
 [ 0.99778269  0.99778269]
 [ 1.9946594   1.9946594 ]
 [ 3.00730595  3.00730595]]

## 牛顿法一步求解最小二乘 
$W_{new} = W - \large\frac{f(W)}{f'(W)}$  

$\normalsize= W - (X^T X)^{-1} \times X^T(XW - Y)$  

$= W - (X^T X)^{-1} \times X^TXW + (X^T X)^{-1} \times X^TY$  

$= W - W + (X^T X)^{-1}X^TY$  

$= (X^T X)^{-1}X^TY$  
  
化简后的结果与最小二乘法的最优解相同

## 利用牛顿法求解根号2的近似值 
目标函数$f(x)$，要求目标函数的最小值，那么$f'(x) = 0$


令$f'(x) = x ^ 2 - 2$

则$f''(x) = 2x$

$x_{n+1} = x_n - \large\frac{f'(x_n)}{f''(x_n)} = x_n - \large\frac{x_n^2-2}{2 x_n} = \large\frac{x_n^2+2}{2 x_n}$

In [ ]:
def square_root(n=2, lr=0.001, threhold=0.0001):
    x = n
    while abs(x ** 2 - 2) > threhold:
        x = (x**2 + 2) / (2 * x)
        print(x)
    return x


In [ ]:
test = square_root(n=2, lr=0.001, threhold=0.0000001)

In [ ]:
1.4142135623746899 ** 2

In [ ]:
print(2 ** 0.5 - test)